# Graphing Recent Changes on a Wiki #

### Table of Contents ###

* [Introduction](#intro)
* [Method and Limitations](#method)
* [Summary Statistics](#stats)
* [Graphs](#graph)


## <a class="anchor" id="intro">Introduction</a> ##



I wanted to determine how frequently users made edits to Talk pages on [Fanlore](www.fanlore.org), a fan-created and fan-run encyclopedia and part of the [Organization for Transformative Works (OTW)](www.transformativeworks.org). I have been an OTW volunteer since 2015 in various roles; having recently become more interested in data analysis and programming, I saw an opportunity to undertake this small project.


Fanlore recruits volunteers called "Gardeners" to handle higher-level editing tasks like moving and deleting pages. Often, users will get the attention of the gardener team by adding a special template to the [Talk page](https://fanlore.org/wiki/Help:Talk_pages) of articles, categories, or files. While exploring possible solutions to helping the volunteers stay on top of changes to these pages and other discussions happening on the site, the question was raised as to the volume of these edits - while it would have been possible to set up some sort of email notification to contact the Gardeners mailing list whenever a Talk page was edited, we would not want to spam the volunteers with notifications. 

###  <a class="anchor" id="method">Method and Limitations</a>

Fanlore runs on the [MediaWiki](https://www.mediawiki.org/wiki/MediaWiki) platform. The software includes an API, which allowed me to pull data from the site directly rather than having to parse an HTML file. I queried the RecentChanges list, specifying that I only wanted the "Talk" namespace.

RecentChanges is an [automatically-generated list](https://www.mediawiki.org/wiki/API:Lists) of every edit made to any page on the Wiki. It aggregates data from the individual edit histories of each page, and therefore it is not typically used as a long-term record. Only 90 days' worth of data is kept at once, and the limit on the number of items returned is 500 for regular users. As such, I was only able to examine edits over the previous 3 months. 

I used the following libraries for this project:

In [ ]:
import requests
import json
from collections import Counter, OrderedDict
import pandas
import plotly.express as px
from datetime import datetime

Using the Requests module, I specified the below parameters to retrieve data from the API. To specify the length of time I wanted to examine, I had to convert 90 days into seconds, as the API did not accept other time formats.

(Computers are really bad at time, y'all....)

In [ ]:
#convert 90 days into seconds - for rcend parameter below
rangeEnd = 60*60*24*90

#dictionary w/ parameters for the API request
parameters = {
    "format": "json", 
    "action": "query", 
    "list": "recentchanges", 
    "rcnamespace": 1, #Talk pages only, excluding User Talk pages
    "rcprop": "timestamp|title|ids|user", #date of edit, title of page, version IDs, and username of editor
    "rcstart": "now", #start of the time range
    "rcend": f"{rangeEnd}", #end of the time range
    "rclimit": 500, #how many edits to return
}

With the parameters set, I could now use the Requests module to ask the API server for data. 
As we specified above, the response was already formatted as a JSON file, but we need a separate library to convert it into a Python object. In this case, it will be formatted as a **dictionary** - an array that contains key-value pairs.

In [ ]:
#ask the Fanlore server for the data and load it with JSON
r = requests.get("https://fanlore.org/w/api.php", params=parameters)
print("Paging fanlore - reponse is: ", r.status_code)
scrape = json.loads(r.content)

"200" means the server accepted our request and is sending over data. 

Now we can loop through the dictionary we created from the JSON data, pulling out the timestamp of each edit and adding it to the `tsList` variable we created to hold the timestamps.

In [ ]:
tsList = []  # Empty list to store the timestamps we will get later

for change in scrape['query']['recentchanges']:
    tsList.append(change['timestamp'])

The timestamps in our list are formatted like this: `2021-06-03T17:10:04Z`
Because we aren't looking at the *time* edits edit were made, only the day, we can use another loop to go through the list and split each timestamp at the "T" character. This will be stored in a new list, called `tsListTrimmed`.

###  <a class="anchor" id="chapter1">High-level Stats</a>

I wanted to also determine the number of total edits, as well as the earliest and latest dates of the edits. Because the data from the API is returned in a chronological order, this was fairly easy to do - we only needed to retrieve the first and last item on the list.

In [ ]:
print(f"Retrieved {len(tsList)} edits from the server. (Maximum is 500)")
print(f"The latest date is {tsList[0]}. The earliest date is {tsList[(len(tsList)-1)]}")

As you can see above, the entries in our list were formatted to include the time of the edit as well. Since I didn't need that info, I split the timestamps at the "T" character, storing the new dates in their own list.

In [ ]:
tsListTrimmed = []

for item in tsList:
    tsListTrimmed.append(item.split('T')[0])

Now that we have a clean list of dates, we can use the Counter object (part of the Collections library) to determine their frequency. This will create a new dictionary with the keys indicating the number of times each value (date) occurs. However, since this dictionary is ordered from most to least frequent, it will not be very useful for plotting on a line graph, so we will convert it to a regular dictionary as well. 

In [ ]:
tsCounter = Counter(tsListTrimmed) # ordered from highest to lowest count...
graphData = dict(tsCounter)  #chronological

Now we are ready to start creating our graph.

In [ ]:
x_val = list(graphData.keys())  # dates
y_val = list(graphData.values())  # freq

print("Making the graph... Check your browser!")

graph = px.line(
    x=x_val,
    y=y_val,
    title="Frequency of Talk Page edits on Fanlore.org, last 90 days",
    labels={"x": "Date", "y": "# of Edits"})

graph.update_xaxes(rangeslider_visible=True)
graph.show()


You can hover over the line to see individual values for each day, or use the slider below the graph to zoom in and isolate a particular range.